<a href="https://colab.research.google.com/github/TutuBraz/avocado-predict-price/blob/main/avocado_price_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo: Criar um modelo que consiga prever o preço do abacate.

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [63]:
df_avocado = pd.read_csv('//content/Avocado.csv')
df_avocado

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27318,2015-11-22,1.97,6249.43,1733.40,2873.92,30.95,1611.16,1590.00,21.16,0.0,organic,2015,WestTexNewMexico
27319,2015-11-29,2.08,4638.10,1395.02,2238.04,61.71,943.33,943.33,0.00,0.0,organic,2015,WestTexNewMexico
27320,2015-12-13,1.80,7836.65,2194.49,2981.01,25.97,2635.18,2598.45,36.73,0.0,organic,2015,WestTexNewMexico
27321,2015-12-20,1.92,6255.19,1512.45,2407.32,11.78,2323.64,2213.72,109.92,0.0,organic,2015,WestTexNewMexico


O df está empilhado pelas regiões

In [57]:
df_avocado.region.unique()

array(['Albany', 'Atlanta', 'Baltimore/Washington', 'BaltimoreWashington',
       'Boise', 'Boston', 'Buffalo/Rochester', 'BuffaloRochester',
       'California', 'Charlotte', 'Chicago', 'Cincinnati/Dayton',
       'CincinnatiDayton', 'Columbus', 'Dallas/Ft. Worth',
       'DallasFtWorth', 'Denver', 'Detroit', 'Grand Rapids',
       'GrandRapids', 'Great Lakes', 'GreatLakes', 'Harrisburg/Scranton',
       'HarrisburgScranton', 'Hartford/Springfield',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'Las Vegas', 'LasVegas', 'Los Angeles', 'LosAngeles', 'Louisville',
       'Miami/Ft. Lauderdale', 'MiamiFtLauderdale', 'Midsouth',
       'Nashville', 'New Orleans/Mobile', 'New York', 'NewOrleansMobile',
       'NewYork', 'Northeast', 'Northern New England',
       'NorthernNewEngland', 'Orlando', 'Philadelphia', 'Phoenix/Tucson',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'Raleigh/Greensboro', 'RaleighGreensboro', 'Richmond/Norfolk',


As regiões aparentemente as regioes estão com valores duplicados. Ex: West Tex/New Mexico e WestTexNewMexico

In [58]:
df_avocado.duplicated().sum()

2

In [59]:
df_avocado.groupby('region').count()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year
region,,,,,,,,,,,,
Albany,506,506,506,506,506,506,506,506,506,506,506,506
Atlanta,506,506,506,506,506,506,506,506,506,506,506,506
Baltimore/Washington,402,402,402,402,402,402,402,402,402,402,402,402
BaltimoreWashington,104,104,104,104,104,104,104,104,104,104,104,104
Boise,506,506,506,506,506,506,506,506,506,506,506,506
...,...,...,...,...,...,...,...,...,...,...,...,...
Total U.S.,402,402,402,402,402,402,402,402,402,402,402,402
TotalUS,104,104,104,104,104,104,104,104,104,104,104,104
West,506,506,506,506,506,506,506,506,506,506,506,506


Observo que as regiões não estão duplicadas e sim estão divididas, as regioes tem que somar um total de 506 valores.

In [66]:
#criando lista com os valores invalido
wrong = []
region_list = list(df_avocado.region.unique())
for i in region_list:
  if '/'  in i or ' ' in i or '.'  in i:
    wrong.append(i)

#criando cópia dos valores errado para substituição 
right = wrong[:]

#alterando valores errado para forma correta
for i in range(len(wrong)): 
  for j in [' ','.','/']:
    right[i] = right[i].replace(j, '')
  
#Criando dicionário que relaciona valores incorreto com corretos
valores = {}
for k in range(len(wrong)):
  valores[wrong[k]]=right[k]

#substituindo valores
df_avocado_new = df_avocado.replace({'region':valores})
     

In [67]:
df_avocado_new.groupby('region').count()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year
region,,,,,,,,,,,,
Albany,506,506,506,506,506,506,506,506,506,506,506,506
Atlanta,506,506,506,506,506,506,506,506,506,506,506,506
BaltimoreWashington,506,506,506,506,506,506,506,506,506,506,506,506
Boise,506,506,506,506,506,506,506,506,506,506,506,506
Boston,506,506,506,506,506,506,506,506,506,506,506,506
BuffaloRochester,506,506,506,506,506,506,506,506,506,506,506,506
California,506,506,506,506,506,506,506,506,506,506,506,506
Charlotte,506,506,506,506,506,506,506,506,506,506,506,506
Chicago,506,506,506,506,506,506,506,506,506,506,506,506


In [11]:
regions = ['Albany', 'Atlanta', 'Baltimore/Washington', 'BaltimoreWashington',
            'Boise', 'Boston', 'Buffalo/Rochester', 'BuffaloRochester',
            'California', 'Charlotte', 'Chicago', 'Cincinnati/Dayton',
            'CincinnatiDayton', 'Columbus', 'Dallas/Ft. Worth',
            'DallasFtWorth', 'Denver', 'Detroit', 'Grand Rapids',
            'GrandRapids', 'Great Lakes', 'GreatLakes', 'Harrisburg/Scranton',
            'HarrisburgScranton', 'Hartford/Springfield',
            'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
            'Las Vegas', 'LasVegas', 'Los Angeles', 'LosAngeles', 'Louisville',
            'Miami/Ft. Lauderdale', 'MiamiFtLauderdale', 'Midsouth',
            'Nashville', 'New Orleans/Mobile', 'New York', 'NewOrleansMobile',
            'NewYork', 'Northeast', 'Northern New England',
            'NorthernNewEngland', 'Orlando', 'Philadelphia', 'Phoenix/Tucson',
            'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
            'Raleigh/Greensboro', 'RaleighGreensboro', 'Richmond/Norfolk',
            'RichmondNorfolk', 'Roanoke', 'Sacramento', 'San Diego',
            'San Francisco', 'SanDiego', 'SanFrancisco', 'Seattle',
            'South Carolina', 'South Central', 'SouthCarolina', 'SouthCentral',
            'Southeast', 'Spokane', 'St. Louis', 'StLouis', 'Syracuse',
            'Tampa', 'Total U.S.', 'TotalUS', 'West', 'West Tex/New Mexico',
            'WestTexNewMexico']

In [12]:
regions_altera = regions[:]

for x in range(len(regions)):
  regions_altera[x] = regions_altera[x].replace("/", "")
  regions_altera[x] = regions_altera[x].replace(" ", "")
  regions_altera[x] = regions_altera[x].replace(".", "")


In [13]:
regions_altera


['Albany',
 'Atlanta',
 'BaltimoreWashington',
 'BaltimoreWashington',
 'Boise',
 'Boston',
 'BuffaloRochester',
 'BuffaloRochester',
 'California',
 'Charlotte',
 'Chicago',
 'CincinnatiDayton',
 'CincinnatiDayton',
 'Columbus',
 'DallasFtWorth',
 'DallasFtWorth',
 'Denver',
 'Detroit',
 'GrandRapids',
 'GrandRapids',
 'GreatLakes',
 'GreatLakes',
 'HarrisburgScranton',
 'HarrisburgScranton',
 'HartfordSpringfield',
 'HartfordSpringfield',
 'Houston',
 'Indianapolis',
 'Jacksonville',
 'LasVegas',
 'LasVegas',
 'LosAngeles',
 'LosAngeles',
 'Louisville',
 'MiamiFtLauderdale',
 'MiamiFtLauderdale',
 'Midsouth',
 'Nashville',
 'NewOrleansMobile',
 'NewYork',
 'NewOrleansMobile',
 'NewYork',
 'Northeast',
 'NorthernNewEngland',
 'NorthernNewEngland',
 'Orlando',
 'Philadelphia',
 'PhoenixTucson',
 'PhoenixTucson',
 'Pittsburgh',
 'Plains',
 'Portland',
 'RaleighGreensboro',
 'RaleighGreensboro',
 'RichmondNorfolk',
 'RichmondNorfolk',
 'Roanoke',
 'Sacramento',
 'SanDiego',
 'SanFrancisc

Problemas ao lidar com data frame:

1.  As PLU não somam o volume total de abacates vendidos.



